In [1]:
# Librairies imports
from geopy.geocoders import Nominatim
import pandas as pd
from openmeteo_py import Hourly,Daily,Options,OWmanager
import boto3
import plotly.express as px
import secret_code_AWS as sec
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import psycopg2
import folium

In [2]:
# CONSTANTS
BUCKET_NAME = "pascal-kayak"

#### CITIES DataFrame construction with :
- a unique id (as primary key for the futur database) : id
- city name : name (from the initial list)
- city latitude : lat (from Nomibatim API)
- city longitude : lon (from Nomibatim API)
- weather score : weather_score (from Open Meteo API, 100 minus the mean of weathercode from 9AM to 18PM during the next 8 days)

#### CITIES DataFrame construction

In [3]:
#
# Constructing the cities dataframe with an id and the name from the list given
#
cities = [
    "Mont Saint Michel",
    "St Malo",
    "Bayeux",
    "Le Havre",
    "Rouen",
    "Paris",
    "Amiens",
    "Lille",
    "Strasbourg",
    "Chateau du Haut Koenigsbourg",
    "Colmar",
    "Eguisheim",
    "Besancon",
    "Dijon",
    "Annecy",
    "Grenoble",
    "Lyon",
    "Gorges du Verdon",
    "Bormes les Mimosas",
    "Cassis",
    "Marseille",
    "Aix en Provence",
    "Avignon",
    "Uzes",
    "Nimes",
    "Aigues Mortes",
    "Saintes Maries de la mer",
    "Collioure",
    "Carcassonne",
    "Ariege",
    "Toulouse",
    "Montauban",
    "Biarritz",
    "Bayonne",
    "La Rochelle",
]
# consruct a dataframe for all the cities with their index
dict_cities = {
    "id": [i + 1 for i in range(len(cities))],
    "name": cities,
    
}

df_cities = pd.DataFrame.from_dict(dict_cities)
df_cities


,id,name
0,1,Mont Saint Michel
1,2,St Malo
2,3,Bayeux
3,4,Le Havre
4,5,Rouen
5,6,Paris
6,7,Amiens
7,8,Lille
8,9,Strasbourg
9,10,Chateau du Haut Koenigsbourg


#### Define the cities geolocation with Nominatim API

In [4]:
#
# Fuction giving the latitude and longitude from a city name
# using the Nominatim Geo Localisation API
#
def lat_lon(city):
    loc = Nominatim(user_agent="GetLoc")

    getLoc = loc.geocode(city)

    # printing address
    print(getLoc.address)

    return (getLoc.latitude, getLoc.longitude)


In [5]:
#
# Constructing 2 colums of latitude and longitude for each city in the dataframe
#
list_lat = []
list_lon = []
for city in df_cities["name"]:
    tmp_lat_lon = lat_lon(city)
    list_lat.append(tmp_lat_lon[0])
    list_lon.append(tmp_lat_lon[1])
df_cities["lat"] = list_lat
df_cities["lon"] = list_lon
df_cities
    

Mont Saint-Michel, Plateforme du Saut-­Gaultier, Le Mont-Saint-Michel, Avranches, Manche, Normandie, France métropolitaine, 50170, France
Saint-Malo, Ille-et-Vilaine, Bretagne, France métropolitaine, 35400, France
Bayeux, Calvados, Normandie, France métropolitaine, 14400, France
Le Havre, Seine-Maritime, Normandie, France métropolitaine, France
Rouen, Seine-Maritime, Normandie, France métropolitaine, France
Paris, Île-de-France, France métropolitaine, France
Amiens, Somme, Hauts-de-France, France métropolitaine, France
Lille, Nord, Hauts-de-France, France métropolitaine, France
Strasbourg, Bas-Rhin, Grand Est, France métropolitaine, France
Château du Haut-Kœnigsbourg, Chemin fermé suite  à travaux, Château du Haut-Kœnigsbourg, Orschwiller, Sélestat-Erstein, Bas-Rhin, Grand Est, France métropolitaine, 67600, France
Colmar, Colmar-Ribeauvillé, Haut-Rhin, Grand Est, France métropolitaine, 68000, France
Eguisheim, Colmar-Ribeauvillé, Haut-Rhin, Grand Est, France métropolitaine, 68420, Fran

,id,name,lat,lon
0,1,Mont Saint Michel,48.635954,-1.511460
1,2,St Malo,48.649518,-2.026041
2,3,Bayeux,49.276462,-0.702474
3,4,Le Havre,49.493898,0.107973
4,5,Rouen,49.440459,1.093966
5,6,Paris,48.858890,2.320041
6,7,Amiens,49.894171,2.295695
7,8,Lille,50.636565,3.063528
8,9,Strasbourg,48.584614,7.750713
9,10,Chateau du Haut Koenigsbourg,48.249490,7.344296


#### Define the cities weather with Open Meteo API

In [6]:
#
# function to retrieve meteo data from Open Meteo API for the next 8 days for a geo localisation
#
def meteo(latitude, longitude):

    hourly = Hourly()
    daily = Daily()
    options = Options(latitude,longitude)

    mgr = OWmanager(options,
        hourly.all(),
        daily.all())

    return mgr.get_data()



In [7]:
#
# Calculating and storing the Weather Score for each city
#

# intialize the list to store the mean weather score for each cities
list_score_weather = []

# taking the meteo data for each cities
for i in df_cities.index:

    # retreving data from Open Meteo API
    meteo_city = meteo(df_cities['lat'][i], df_cities['lon'][i])

    # keeping the data hour per hour in a dataframe
    df_meteo = pd.DataFrame.from_dict(meteo_city["hourly"])

    # constructing a colonne only with hours (the 5 last characters)
    df_meteo["hour"] = df_meteo.apply(lambda row : row["time"][-5:], axis = 1)

    # kepping only meteo data in the day hours (from 9AM to 18PM)
    mask_day_hours = (df_meteo["hour"] > "08:00") & (df_meteo["hour"] < "19:00")
    df_meteo = df_meteo.loc[mask_day_hours, ["weathercode"]]

    # defining the score_weather by substracting the average weather code 
    # from 9AM to 18PM to 100 to have the greatest number for the good weather
    list_score_weather.append(100 - df_meteo["weathercode"].mean())


# adding the lists of meteo data into the cities dataframe
df_cities["score_weather"] = list_score_weather
df_cities



,id,name,lat,lon,score_weather
0,1,Mont Saint Michel,48.635954,-1.511460,92.985714
1,2,St Malo,48.649518,-2.026041,92.871429
2,3,Bayeux,49.276462,-0.702474,98.028571
3,4,Le Havre,49.493898,0.107973,97.042857
4,5,Rouen,49.440459,1.093966,96.242857
5,6,Paris,48.858890,2.320041,94.314286
6,7,Amiens,49.894171,2.295695,92.728571
7,8,Lille,50.636565,3.063528,97.442857
8,9,Strasbourg,48.584614,7.750713,88.614286
9,10,Chateau du Haut Koenigsbourg,48.249490,7.344296,81.300000


In [8]:
#
# saving the cities dataframe to a csv file
#
df_cities.to_csv('cities.csv', index=False)

#### Upload the Cities and Hotels CSV files in a AWS S3 Bucket

In [9]:
# Opening a AWS session with SECRET IDs from the secret_code file
session = boto3.Session(aws_access_key_id=sec.AWS_ACCESS_KEY_ID, 
                        aws_secret_access_key=sec.AWS_SECRET_ACCESS_KEY)
# Creating a S3 bucket to store the csv files
s3 = session.resource("s3")

s3.meta.client.upload_file('./cities.csv', BUCKET_NAME, 'cities.csv')
# Uploading the csv file for the hotels scrapped with scrappy
s3.meta.client.upload_file('./hotels.csv', BUCKET_NAME, 'hotels.csv')


#### Download the Cities and Hotels CSV Files from the AWS S3 Bucket and storing them in a RDS Postgre DataBase
- the cities data will be a table
- the hotels data will be a table (without the city_name, useless because of the hotel_id)

In [10]:
# Downloading the CSV Files from the AWS S3 Bucket stored in a specifique folder (csv_from_aws)
session = boto3.Session(aws_access_key_id=sec.AWS_ACCESS_KEY_ID, 
                        aws_secret_access_key=sec.AWS_SECRET_ACCESS_KEY)

session.resource('s3').Bucket(BUCKET_NAME).download_file('cities.csv','./csv_from_aws/cities.csv')
session.resource('s3').Bucket(BUCKET_NAME).download_file('hotels.csv','./csv_from_aws/hotels.csv')

In [11]:
df_cities = pd.read_csv('./csv_from_aws/cities.csv')
df_hotels = pd.read_csv('./csv_from_aws/hotels.csv')
print(df_cities.head())
print(df_hotels.head())

   id               name        lat       lon  score_weather
0   1  Mont Saint Michel  48.635954 -1.511460      92.985714
1   2            St Malo  48.649518 -2.026041      92.871429
2   3             Bayeux  49.276462 -0.702474      98.028571
3   4           Le Havre  49.493898  0.107973      97.042857
4   5              Rouen  49.440459  1.093966      96.242857
   city_id   city_name                                        name  \
0        7      Amiens                       WHITE HOUSE DHAVERNAS   
1        9  Strasbourg  Carpe Diem Home - Au pied de la Cathédrale   
2        6       Paris      Citadines Saint-Germain-des-Prés Paris   
3        3      Bayeux                          LOGIS DU GRAND PIN   
4        8       Lille                  Novotel Lille Centre Gares   

                                        booking_link  score        lat  \
0  https://www.booking.com/hotel/fr/white-house-d...    8.7  49.881470   
1  https://www.booking.com/hotel/fr/carpe-diem-ho...    9.5  48.5

In [12]:
#
# Deleting the tables CITIES and HOTELS if they already exist
#
dbconnection = psycopg2.connect(
   database="postgres", user=sec.RDS_USERNAME, password=sec.RDS_PASSWORD, host=sec.RDS_HOSTNAME, port= '5432'
)
cursor = dbconnection.cursor()

#Dropping EMPLOYEE table if already exists
cursor.execute("DROP TABLE IF EXISTS CITIES")
print("Table CITIES dropped")

cursor.execute("DROP TABLE IF EXISTS HOTELS")
print("Table HOTELS dropped")

#Commit your changes in the database
dbconnection.commit()

#Closing the connection
dbconnection.close()

Table CITIES dropped
Table HOTELS dropped


In [13]:
#
# dropping the city_name column from the hotels dataframe to avoid duplicate data (the city name already in in the CITIES table and can be join with the city_id)
#
df_hotels = df_hotels.drop(columns="city_name")
df_hotels.head()

,city_id,name,booking_link,score,lat,lon,descr
0,7,WHITE HOUSE DHAVERNAS,https://www.booking.com/hotel/fr/white-house-d...,8.7,49.881470,2.299871,"Situé à Amiens, en Picardie, à proximité de la..."
1,9,Carpe Diem Home - Au pied de la Cathédrale,https://www.booking.com/hotel/fr/carpe-diem-ho...,9.5,48.582077,7.749161,"Proposant des hébergements avec une cuisine, l..."
2,6,Citadines Saint-Germain-des-Prés Paris,https://www.booking.com/hotel/fr/citadines-apa...,8.1,48.854926,2.341155,Le Citadines Saint-Germain-des-Prés Paris béné...
3,3,LOGIS DU GRAND PIN,https://www.booking.com/hotel/fr/logis-du-gran...,9.5,49.252004,-0.669848,"Offrant une vue sur le jardin, le LOGIS DU GRA..."
4,8,Novotel Lille Centre Gares,https://www.booking.com/hotel/fr/novotel-lille...,8.0,50.634441,3.073358,Situé à seulement 300 mètres de la gare de Lil...


In [14]:
# Creating a session to store the csv files on the postgresql RDS database
engine = create_engine(f"postgresql+psycopg2://{sec.RDS_USERNAME}:{sec.RDS_PASSWORD}@{sec.RDS_HOSTNAME}/postgres", echo=True)
Session = sessionmaker(bind=engine)
df_cities.to_sql("cities", engine, index=False)
df_hotels.to_sql("hotels", engine, index=False)


2023-01-30 09:19:33,094 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-01-30 09:19:33,096 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-01-30 09:19:33,108 INFO sqlalchemy.engine.Engine select current_schema()
2023-01-30 09:19:33,110 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-01-30 09:19:33,122 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-01-30 09:19:33,124 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-01-30 09:19:33,138 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2023-01-30 09:19:33,139 INFO sqlalchemy.engine.Engine [generated in 0.00117s] {'name': 'cities'}
2023-01-30 09:19:33,159 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-30 09:19:33,161 INFO sqlalchemy.engine.Engine 
CREATE TABLE cities (
	id BIGINT, 
	name TEXT, 
	lat FLOAT(53), 
	lon FLOAT(53), 
	score_weather FLOAT(53)
)


2023-01-30 09:19:33,162 

875

#### Load the Cities and Hotels Data into DataFrames from the RDS Database

In [15]:
sql = "SELECT id, name, lat, lon, score_weather FROM cities"

engine = create_engine(f"postgresql+psycopg2://{sec.RDS_USERNAME}:{sec.RDS_PASSWORD}@{sec.RDS_HOSTNAME}/postgres", echo=True)
Session = sessionmaker(bind=engine)
dbConnection = engine.connect()


# Reading the Cities Data
df_cities  = pd.read_sql(sql, dbConnection)


# Print the DataFrame
print(df_cities)


 
# Close the database connection
dbConnection.close()

2023-01-30 09:19:33,537 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-01-30 09:19:33,538 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-01-30 09:19:33,550 INFO sqlalchemy.engine.Engine select current_schema()
2023-01-30 09:19:33,550 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-01-30 09:19:33,562 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-01-30 09:19:33,563 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-01-30 09:19:33,576 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2023-01-30 09:19:33,576 INFO sqlalchemy.engine.Engine [generated in 0.00073s] {'name': 'SELECT id, name, lat, lon, score_weather FROM cities'}
2023-01-30 09:19:33,588 INFO sqlalchemy.engine.Engine SELECT id, name, lat, lon, score_weather FROM cities
2023-01-30 09:19:33,588 INFO sqlalchemy.engine.Engine [raw sql] {}
    id                          name     

#### Show the map of the cities

In [16]:
fig = px.scatter_mapbox(
        df_cities, 
        lat="lat", 
        lon="lon",
        text="name",
        hover_name="name",
        size="score_weather", 
        color="score_weather",
        width=1000,
        height=800,
        mapbox_style="open-street-map"
)

fig.show()

#### Choose the 5 top cities

In [17]:
# Selecting the 5 top cities regarding the best weather score
df_cities.sort_values(by="score_weather", ascending=False, inplace=True, ignore_index=True)
df_cities_good_weather = df_cities.loc[0:4, :]
df_cities_good_weather[["name", "score_weather"]].head(5)

,name,score_weather
0,Aix en Provence,99.900000
1,Ariege,99.885714
2,Cassis,99.785714
3,Collioure,99.757143
4,Marseille,99.728571


In [18]:
# variables to center the map on the 5 cities
lat_mean = df_cities_good_weather["lat"].mean()
lon_mean = df_cities_good_weather["lon"].mean()

In [19]:
# Connecting to the database 
engine = create_engine(f"postgresql+psycopg2://{sec.RDS_USERNAME}:{sec.RDS_PASSWORD}@{sec.RDS_HOSTNAME}/postgres", echo=True)
Session = sessionmaker(bind=engine)
dbConnection = engine.connect()

# for each top 5 city, taking the 20 best hotels
i = 0
for city_id in df_cities_good_weather["id"]:
    sql = f"""
    SELECT h.name, h.lat, h.lon, h.booking_link, h.score, h.descr
    FROM hotels h, cities c
    WHERE h.city_id = c.id
    AND  h.city_id = {city_id} ;
    """
    df = pd.read_sql(sql, dbConnection)
    df.sort_values(by="score", ascending=False, inplace=True, ignore_index=True)
    
    # building the 20 hotels dataframe
    if i == 0:
        df_hotels_good_weather = df.loc[:19,:]
    else:
        df = df.loc[:19,:]
        df_hotels_good_weather = pd.concat([df_hotels_good_weather, df], ignore_index=True)
    i += 1

# Close the database connection
dbConnection.close()   

2023-01-30 09:19:34,236 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-01-30 09:19:34,238 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-01-30 09:19:34,250 INFO sqlalchemy.engine.Engine select current_schema()
2023-01-30 09:19:34,252 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-01-30 09:19:34,264 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-01-30 09:19:34,266 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-01-30 09:19:34,279 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2023-01-30 09:19:34,280 INFO sqlalchemy.engine.Engine [generated in 0.00071s] {'name': '\n    SELECT h.name, h.lat, h.lon, h.booking_link, h.score, h.descr\n    FROM hotels h, cities c\n    WHERE h.city_id = c.id\n    AND  h.city_id = 22 ;\n    '}
2023-01-30 09:19:34,296 INFO sqlalchemy.engine.Engine 
    SELECT h.name, h.lat, h.lon, h.booking_link, h.scor

In [20]:
df_hotels_good_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          100 non-null    object 
 1   lat           100 non-null    float64
 2   lon           100 non-null    float64
 3   booking_link  100 non-null    object 
 4   score         100 non-null    float64
 5   descr         100 non-null    object 
dtypes: float64(3), object(3)
memory usage: 4.8+ KB


In [21]:
df_hotels_good_weather[["name", "score"]].head(20)

,name,score
0,Les Suites du Cours & Spa,9.1
1,N.8 LIFESTYLE SUITES,9.1
2,T2 classé 3 étoile au centre du quartier histo...,8.7
3,Aparthotel Adagio Aix-en-Provence Centre,8.4
4,Domaine Gaogaia,8.4
5,Hotel Cardinal,8.4
6,Aquabella Hôtel & Spa,8.4
7,Hôtel Escaletto,8.3
8,Hôtel Le Mozart,8.3
9,Renaissance Aix-en-Provence Hotel,8.2


In [22]:
m = folium.Map(location=[lat_mean, lon_mean], zoom_start=8)


In [23]:
df_hotels_good_weather.apply(
    lambda row:folium.Marker(
        location=[row["lat"], 
        row["lon"]], 
        popup="<a href='" + row['booking_link'] + "'>" + row['name'] + "</a>" + " " + str(row['score']))
        .add_to(m), axis=1)
m